In [2]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: ^C
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/huggingface_hub/commands/huggingface_cli.py", line 41, in main
    service.run()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/huggingface_

In [ ]:
!pip3 install wandb

In [ ]:
import wandb

wandb.login()

In [ ]:
%env WANDB_LOG_MODEL=true

## Download Data (Pre-processed 230,623 spike glycoprotein sequences stored on github)

In [ ]:
!wget https://media.githubusercontent.com/media/hunarbatra/covid-spike-data/master/spike-gisaid.txt

In [ ]:
!pip install git+https://github.com/huggingface/transformers

## Tokenizer Training

In [ ]:
%%time 
from pathlib import Path
import os
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("../transformer_data/").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=20, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

vocab_size = tokenizer.get_vocab_size()
print(vocab_size)
print(tokenizer.get_vocab())

In [ ]:
!mkdir CoVBERT
tokenizer.save_model("CoVBERT")

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./CoVBERT/vocab.json",
    "./CoVBERT/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=2048)

In [ ]:
tokenizer.encode("MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHVIHVSGTNGTKR")

In [ ]:
tokenizer.encode("MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHVIHVSGTNGTKR").tokens

# CoVBERT training

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=vocab_size,
    max_position_embeddings=2048,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./CoVBERT", max_len=2048)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 44 million parameters

44895237

Building the training set

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./spike-gisaid.txt",
    block_size=1389,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 4min 41s, sys: 33.6 s, total: 5min 14s
Wall time: 55.1 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
dataset = dataset[:50000]

In [ ]:
# Split dataset X% train, (1-X)% test ----- > 85% train, 15% test
X = 80
train_csv = dataset[int((1-float(X)/100) * len(dataset)):]
eval_csv = dataset[:int((1-float(X)/100) * len(dataset))]
print(f"# of training samples:  {len(train_csv)}")
print(f'# of evaluation samples: {len(eval_csv)}')

# of training samples:  40001
# of evaluation samples: 9999


In [ ]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    report_to = 'wandb',  # enable logging to W&B
    run_name = 'covbert_training', # name of the W&B run
    output_dir="./CoVBERT",
    evaluation_strategy = 'steps', 
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8, # crashes with 16, ... 64...
    # per_gpu_train_batch_size=64,
    logging_steps = 100, # we will log every 100 steps
    save_steps=100,
    save_total_limit=2,
    prediction_loss_only=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_csv ,
    eval_dataset=eval_csv,
    # compute_metrics=compute_metrics,
)

# Training

In [ ]:
%%time
trainer.train()

Validation

In [ ]:
trainer.evaluate()

Save model

In [ ]:
trainer.save_model("./CoVBERT)

Push model to HuggingFace

In [ ]:
trainer.push_to_hub()

In [ ]:
wandb.finish()